In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import pandas as pd
import numpy as np
import json
import torch
import re
import ast

In [ ]:
model_name = "Qwen/Qwen2.5-14B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="mps"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [ ]:
system_prompt = "You are an expert reasoning assistant. Your goal is to answer factual yes/no questions using clear, logical, and multi-step reasoning. Always provide the reasoning first, then a final answer."

user_prompt_template = """
{question}

Answer the question step-by-step using logical reasoning based on commonsense or factual knowledge.

Format your response as:
Reason: [step 1: ..., step 2: ..., ..., step n: ...]
Answer: [True/False]
"""


In [ ]:
with open('../strategyQA/strategyqa.json', 'r') as f:
    data = json.load(f)

In [ ]:
data[0]

In [ ]:


datapoint = data[11]
ques = datapoint['question']
gt_answer = datapoint['answer']

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_template.format(question=ques)}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response


In [ ]:

def save_pred(id_, response, ques, gt_answer, path):
    response = response.replace('"', "'")
    reason_start = re.search(r"Reason:", response, re.DOTALL).span()[0]
    reason_end = re.search(r"Reason:", response, re.DOTALL).span()[1]
    answer_start = re.search(r"Answer:", response, re.DOTALL).span()[0]
    answer_end = re.search(r"Answer:", response, re.DOTALL).span()[1]
    
    reason_text = response[reason_start:answer_start]
    reason_lines = [line.strip() for line in reason_text.strip().split("\n") if line.strip()]
    
    answer_text = response[answer_start:]
    answer_str = answer_text.replace("Answer:", "").strip()
    
    result = {
            "id": id_,
            "Question": ques,
            "Reason": reason_lines,
            "Answer": answer_str,
            "GT_answer": gt_answer,
        }
    with open(path, "w") as f:
        json.dump(result, f, indent=2)


In [ ]:

for num, datapoint in enumerate(data):
    #if num < 809:
    #    continue
    print(num)

    ques = datapoint['question']
    gt_answer = datapoint['answer']

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_template.format(question=ques)}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    path = '../strategyqa/qwen_25_pred/' + str(num) + '.json'
    save_pred(datapoint['qid'], response, ques, gt_answer, path)
    
    


In [ ]:
response

In [ ]:

response = response.replace('"', "'")
reason_start = re.search(r"Reason:", response, re.DOTALL).span()[0]
reason_end = re.search(r"Reason:", response, re.DOTALL).span()[1]
answer_start = re.search(r"Answer:", response, re.DOTALL).span()[0]
answer_end = re.search(r"Answer:", response, re.DOTALL).span()[1]

reason_text = response[reason_start:answer_start]
reason_lines = [line.strip() for line in reason_text.strip().split("\n") if line.strip()]

answer_text = response[answer_start:]
answer_str = answer_text.replace("Answer:", "").strip()

result = {
        "Question": ques,
        "Reason": reason_lines,
        "Answer": answer_str,
        "GT_answer": gt_answer,
    }
with open('../1.json', "w") as f:
    json.dump(result, f, indent=2)
